Split daily Reddit Parquet shards by subreddit and re-upload.

In [7]:
!pip install -q pyarrow fastparquet
!pip install -U huggingface_hub

  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.30.2
    Uninstalling huggingface-hub-0.30.2:
      Successfully uninstalled huggingface-hub-0.30.2


In [11]:
from __future__ import annotations

import os
import re
import shutil
import tempfile
from pathlib import Path
from typing import Iterable

import pandas as pd
from huggingface_hub import HfApi, hf_hub_download, CommitOperationAdd
from tqdm.auto import tqdm

from dotenv import load_dotenv

load_dotenv()

True

In [12]:
def _sanitize(name: str) -> str:
    """
    Make subreddit safe for filenames (removes slashes, spaces, etc.).
    """
    name = name.strip().lower()
    name = re.sub(r"[^\w\-\.]", "_", name)  # keep letters, numbers, _, -, .
    return name
    
def split_and_upload_by_subreddit(
    repo_id: str = "hblim/top_reddit_posts_daily",
    source_folder: str = "data_scored",
    target_folder: str = "data_scored_subreddit",
    overwrite: bool = False,
    batch_size: int = 20,
    token: str | None = None,
) -> None:
    """
    For every Parquet in `source_folder`, create one Parquet per subreddit
    and upload to `target_folder`.

    Parameters
    ----------
    repo_id : str
        Hugging Face dataset repo id.
    source_folder : str
        Folder that already contains the daily Parquet files.
    target_folder : str
        New folder to hold subreddit-level Parquet shards.
    overwrite : bool
        Re-process / re-upload even if the target file already exists.
    batch_size : int
        Upload this many files per commit (reduces commit spam).
    token : str | None
        HF token; if None, uses the one stored by `huggingface-cli login`.
    """
    api = HfApi(token=token)

    # 1. discover daily Parquet files in the repo
    files_in_repo: Iterable[str] = api.list_repo_files(repo_id, repo_type="dataset")
    daily_files = sorted(
        f for f in files_in_repo if f.startswith(source_folder) and f.endswith(".parquet")
    )
    if not daily_files:
        raise RuntimeError(f"No Parquet files found in {source_folder}")

    print(f"Found {len(daily_files)} daily shards in {source_folder}")

    with tempfile.TemporaryDirectory() as tmp_dir:
        tmp_dir = Path(tmp_dir)

        upload_queue: list[tuple[Path, str]] = []
        pbar = tqdm(daily_files, desc="processing days", unit="file")

        for remote_path in pbar:
            file_date = Path(remote_path).stem  # e.g. 2025-05-31
            local_path = hf_hub_download(
                repo_id=repo_id,
                filename=remote_path,
                repo_type="dataset",
                cache_dir=tmp_dir,  # keep inside temp dir
            )
            df = pd.read_parquet(local_path)

            # 2. split by subreddit
            for subreddit, sub_df in df.groupby("subreddit", sort=False):
                safe_sub = _sanitize(subreddit)
                out_fname = f"{file_date}__{safe_sub}.parquet"
                out_repo_path = f"{target_folder}/{out_fname}"

                # skip if already in repo and not overwriting
                if not overwrite and out_repo_path in files_in_repo:
                    continue

                out_local = tmp_dir / out_fname
                sub_df.to_parquet(out_local, index=False)
                upload_queue.append((out_local, out_repo_path))

            # upload in batches to reduce commit churn
            if len(upload_queue) >= batch_size:
                _flush_upload_queue(api, repo_id, upload_queue)
                upload_queue.clear()

        # flush any leftovers
        if upload_queue:
            _flush_upload_queue(api, repo_id, upload_queue)

    print("✅ Done – all subreddit shards uploaded.")


def _flush_upload_queue(api: HfApi, repo_id: str,
                        queue: list[tuple[Path, str]]) -> None:
    """Upload a batch of files in one commit (works on ≥0.28)."""
    if not queue:
        return

    ops = [
        CommitOperationAdd(
            path_in_repo=dst,         # where the file will live in the repo
            path_or_fileobj=str(src)  # local temp file
        )
        for src, dst in queue
    ]

    api.create_commit(
        repo_id=repo_id,
        repo_type="dataset",
        operations=ops,
        commit_message=f"Add {len(queue)} subreddit parquet file(s)",
    )

In [13]:
# Example call – adjust repo_id / token as needed
split_and_upload_by_subreddit(
    repo_id="hblim/top_reddit_posts_daily",
    source_folder="data_scored",
    target_folder="data_scored_subreddit",
    overwrite=False,  # set True if you need to regenerate everything
    batch_size=50,    # tweak for faster / slower commits
)

Found 35 daily shards in data_scored


processing days:   0%|          | 0/35 [00:00<?, ?file/s]

2025-05-01.parquet:   0%|          | 0.00/271k [00:00<?, ?B/s]

2025-05-02.parquet:   0%|          | 0.00/202k [00:00<?, ?B/s]

2025-05-03.parquet:   0%|          | 0.00/231k [00:00<?, ?B/s]

...

2025-06-04__artificial.parquet:   0%|          | 0.00/35.2k [00:00<?, ?B/s]

2025-06-04__localllama.parquet:   0%|          | 0.00/84.6k [00:00<?, ?B/s]

2025-06-04__singularity.parquet:   0%|          | 0.00/84.3k [00:00<?, ?B/s]

2025-06-04__openai.parquet:   0%|          | 0.00/88.2k [00:00<?, ?B/s]

✅ Done – all subreddit shards uploaded.
